# Download dependencies

In [ ]:
#https://github.com/ckiplab/ckiptagger/wiki/Chinese-README

from ckiptagger import data_utils, construct_dictionary, WS, POS, NER

# 下載模型
data_utils.download_data_gdown("./")

ws = WS("./data")

# Read Data

In [ ]:
import json

with open('./wiki_2021_10_05_50000.json', 'r', encoding="utf-8") as f:
    data = json.load(f)

print(data[0])


In [ ]:
# Preview
test_text = data[0]['articles']
ws([test_text])


# Config

In [ ]:
import sqlite3

database = './table2.db'
conn = sqlite3.connect(database)
cursor = conn.cursor()

delimiter_set = {",", "，",
                 ".", "。",
                 ":", "：",
                 "！", "!",
                 "；", ";",
                 "?", "？",
                 "（", "）",
                 "(", ")",
                 "\"", "'"}


# Running

In [ ]:
from tqdm import tqdm

batch_size = 100
for index in tqdm(range(len(data) // batch_size)):
    input = []
    for i in range(batch_size):
        input.append(data[index * batch_size + i]['articles'])
        input.append(data[index * batch_size + i]['title'])

    text = ws(input, segment_delimiter_set=delimiter_set)

    for i in range(batch_size):
        for key in text[i*2] + text[i*2+1]:
            if key not in delimiter_set:
                cursor.execute('INSERT INTO inverted_table(term, id) VALUES(?, ?)',
                               (key, data[index * batch_size + i]['id']))
    conn.commit()


In [ ]:

cursor.close()
conn.close()